# PageRank

In [1]:
def f(x):
    print x
    list1=[]
    s=len(x[1][0])
    for y in x[1][0]:
        list1.append(tuple((y,x[1][1]/s)))
    return list1

def p(x):
    print x

In [4]:
list=[('A',('D',)),('B',('A',)),('C',('A','B')),('D',('A','C'))]
pages=sc.parallelize(list).map(lambda x:(x[0],tuple(x[1]))).partitionBy(2).cache()#必须转换成key-values,持久化操作提高效率，partitionBy将相同key的元素哈希到相同的机器上，省去了后续join操作shuffle开销
links=sc.parallelize(['A','B','C','D']).map(lambda x:(x,1.0))
for i in range(1,3):
    rank=pages.join(links).flatMap(f)
    links=rank.reduceByKey(lambda x, y:x+y)
    links=links.mapValues(lambda x:0.15+0.85*x)
    links.foreach(p)
#
links.saveAsSequenceFile("pagerank.output")